In [1]:
import pandas as pd
import numpy as np
import ccxt
import time
import datetime as dt
from datetime import datetime
import time

# Binance API data

In [11]:
binance = ccxt.binanceusdm()
coin_name = 'BTC/USDT'

# fetchOHLCV 매서드 ver.2
# fetchOHLCV ('BTC/USDT', timeframe = '1m', since = 'undefined', limit = 'undefined', params = {})
dateSet = []
def load_coin_data(symbol, time_m, date) : 
    dateSet.append(date)
    date = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')  #타입 변환
    dt_utc = date + dt.timedelta(hours=-9)  #KST -> UTC

    dt_utc = dt_utc.strftime('%Y-%m-%d %H:%M:%S')  #datetime -> str
    timestamp = time.mktime(datetime.strptime(dt_utc, '%Y-%m-%d %H:%M:%S').timetuple())*1000  #stamp형태로 변환

    BTC_data = binance.fetch_ohlcv(symbol, time_m, since = round(timestamp))  #코인명, 코인시세 간격 입력
    df_BTC = pd.DataFrame(BTC_data, columns = ['datetime','Open','High','Low','Close','Volume'])
    
    df_BTC['datetime'] = pd.to_datetime(df_BTC['datetime'], unit='ms')
    
    for i in range(len(df_BTC)) :
        df_BTC.loc[i,'datetime'] = df_BTC.loc[i,'datetime'] + dt.timedelta(hours=9)  #UTC -> KST    

    # df_BTC.set_index('datetime', inplace=True)

    return df_BTC

In [12]:
coin_data1 = load_coin_data(coin_name, '1h', '2022-12-01 00:00:00')
coin_data2 = load_coin_data(coin_name, '1h', '2022-12-10 00:00:00')
print(coin_data1.head(4))
print(coin_data2.tail(4))
binanace_coin_20221226 = pd.concat([coin_data1, coin_data2], axis = 0)
binanace_coin_20221226 = binanace_coin_20221226.drop_duplicates(['datetime'])
binanace_coin_20221226

             datetime     Open     High      Low    Close     Volume
0 2022-11-30 15:00:00  16858.0  16863.6  16820.0  16854.1  10934.682
1 2022-11-30 16:00:00  16854.0  16916.3  16852.4  16874.1  11913.728
2 2022-11-30 17:00:00  16874.1  16926.2  16865.0  16883.7  11360.114
3 2022-11-30 18:00:00  16883.7  16898.0  16830.0  16870.8  11606.671
               datetime     Open     High      Low    Close    Volume
414 2022-12-26 21:00:00  16854.1  16858.9  16844.7  16849.9  3250.590
415 2022-12-26 22:00:00  16849.9  16850.6  16821.0  16829.9  6656.667
416 2022-12-26 23:00:00  16830.0  16836.2  16817.0  16823.6  5418.780
417 2022-12-27 00:00:00  16823.6  16828.4  16823.6  16827.5  1080.094


,datetime,Open,High,Low,Close,Volume
0,2022-11-30 15:00:00,16858.0,16863.6,16820.0,16854.1,10934.682
1,2022-11-30 16:00:00,16854.0,16916.3,16852.4,16874.1,11913.728
2,2022-11-30 17:00:00,16874.1,16926.2,16865.0,16883.7,11360.114
3,2022-11-30 18:00:00,16883.7,16898.0,16830.0,16870.8,11606.671
4,2022-11-30 19:00:00,16870.7,16891.7,16858.4,16858.4,6739.155
...,...,...,...,...,...,...
413,2022-12-26 20:00:00,16840.4,16863.0,16836.8,16854.0,4454.314
414,2022-12-26 21:00:00,16854.1,16858.9,16844.7,16849.9,3250.590
415,2022-12-26 22:00:00,16849.9,16850.6,16821.0,16829.9,6656.667
416,2022-12-26 23:00:00,16830.0,16836.2,16817.0,16823.6,5418.780


In [13]:
binanace_coin_20221226.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634 entries, 0 to 417
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  634 non-null    datetime64[ns]
 1   Open      634 non-null    float64       
 2   High      634 non-null    float64       
 3   Low       634 non-null    float64       
 4   Close     634 non-null    float64       
 5   Volume    634 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 34.7 KB


In [14]:
#바이낸스 데이터 변환
binanace_coin_20221226 = binanace_coin_20221226.astype({'datetime' : 'str'})
binanace_coin_20221226['Date'] = binanace_coin_20221226['datetime'].str.contains('2022-12')
binanace_coin_20221226 = binanace_coin_20221226[(binanace_coin_20221226['Date'] == True)]
binanace_coin_20221226 = binanace_coin_20221226.drop(["Date"], axis = 1)
binanace_coin_20221226 = binanace_coin_20221226.reset_index(drop=True)
binanace_coin_20221226

,datetime,Open,High,Low,Close,Volume
0,2022-12-01 00:00:00,16838.5,16860.9,16789.0,16856.8,12557.733
1,2022-12-01 01:00:00,16856.8,16899.1,16820.2,16853.4,17208.495
2,2022-12-01 02:00:00,16853.3,16869.7,16710.3,16782.1,26428.364
3,2022-12-01 03:00:00,16782.0,16977.6,16700.0,16913.4,62864.217
4,2022-12-01 04:00:00,16913.4,17138.7,16911.0,17051.5,63307.753
...,...,...,...,...,...,...
620,2022-12-26 20:00:00,16840.4,16863.0,16836.8,16854.0,4454.314
621,2022-12-26 21:00:00,16854.1,16858.9,16844.7,16849.9,3250.590
622,2022-12-26 22:00:00,16849.9,16850.6,16821.0,16829.9,6656.667
623,2022-12-26 23:00:00,16830.0,16836.2,16817.0,16823.6,5418.780


# OneBot API data

In [17]:
import requests
import json

# Request
requestData = requests.get(' URL ')
print(requestData)

if requestData.status_code == 200 :
    print('Request OK')
    jsonData = requestData.json()

# 데이터 변환
onebot_coin_score_20221226 = pd.DataFrame(jsonData)
onebot_coin_score_20221226 = onebot_coin_score_20221226.drop(["info"], axis = 1)
onebot_coin_score_20221226 = onebot_coin_score_20221226.transpose()
onebot_coin_score_20221226 = onebot_coin_score_20221226[['date', 'score']]
onebot_coin_score_20221226

<Response [200]>
Request OK


,date,score
0,2022-11-27 14:02:49,39.2
1,2022-11-27 15:02:47,67.3
2,2022-11-27 17:02:48,77.5
3,2022-11-27 18:02:49,77.5
4,2022-11-27 19:02:48,137.9
...,...,...
693,2022-12-26 20:01:52,84.7
694,2022-12-26 21:01:55,85.7
695,2022-12-26 22:01:54,84.7
696,2022-12-26 23:01:53,75.2


In [18]:
#원봇 데이터 변환
onebot_coin_score_20221226['Date'] = onebot_coin_score_20221226['date'].str.contains('2022-12')
onebot_coin_score_20221226 = onebot_coin_score_20221226[(onebot_coin_score_20221226['Date'] == True)]
onebot_coin_score_20221226 = onebot_coin_score_20221226.drop(["Date"], axis = 1)
onebot_coin_score_20221226 = onebot_coin_score_20221226.reset_index(drop=True)
onebot_coin_score_20221226

,date,score
0,2022-12-01 00:01:50,15.8
1,2022-12-01 01:01:49,15.2
2,2022-12-01 02:01:46,12
3,2022-12-01 03:01:48,-12.5
4,2022-12-01 04:01:53,11.8
...,...,...
619,2022-12-26 20:01:52,84.7
620,2022-12-26 21:01:55,85.7
621,2022-12-26 22:01:54,84.7
622,2022-12-26 23:01:53,75.2


In [19]:
# 데이터 합치기
coin_data_20221226 = pd.concat([binanace_coin_20221226, onebot_coin_score_20221226], axis = 1)
coin_data_20221226 = coin_data_20221226.drop(["date"], axis = 1)
# total_data_20221226 = total_data_20221226.drop(["date"], axis = 1) # 인덱스 넘버로 행 삭제
coin_data_20221226

,datetime,Open,High,Low,Close,Volume,score
0,2022-12-01 00:00:00,16838.5,16860.9,16789.0,16856.8,12557.733,15.8
1,2022-12-01 01:00:00,16856.8,16899.1,16820.2,16853.4,17208.495,15.2
2,2022-12-01 02:00:00,16853.3,16869.7,16710.3,16782.1,26428.364,12
3,2022-12-01 03:00:00,16782.0,16977.6,16700.0,16913.4,62864.217,-12.5
4,2022-12-01 04:00:00,16913.4,17138.7,16911.0,17051.5,63307.753,11.8
...,...,...,...,...,...,...,...
620,2022-12-26 20:00:00,16840.4,16863.0,16836.8,16854.0,4454.314,85.7
621,2022-12-26 21:00:00,16854.1,16858.9,16844.7,16849.9,3250.590,84.7
622,2022-12-26 22:00:00,16849.9,16850.6,16821.0,16829.9,6656.667,75.2
623,2022-12-26 23:00:00,16830.0,16836.2,16817.0,16823.6,5418.780,85.7


In [21]:
# 파일 저장
from openpyxl import Workbook

coin_data_20221226.to_excel('BTC_data_20221226.xlsx') #excel파일로 생성